In [1]:
from bs4 import BeautifulSoup
import requests as rq
import pandas as pd
import numpy as np


from urllib.parse import urljoin

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By

import time

In [2]:
# Defining all the things that are to be scraped
global rating, Aircraft, Class, Travel_type, Route, Date, comfort, staff, food, entertainment, wifi, ground_service, vfm, recommend, airline_name

verified=[]
rating=[]
feedback=[]
Aircraft=[]
Class=[]
Travel_type=[]
Route=[]
Date=[]
comfort=[]
staff=[]
food=[]
entertainment=[]
wifi=[]
ground_service=[]
vfm=[]
recommend=[]
airline_name=[]

In [3]:
# This function extracts all the stars from all the different rating categories

def value_fetcher(soup):
    rating.append(np.nan)
    Aircraft.append(np.nan)
    Class.append(np.nan)
    Travel_type.append(np.nan)
    Route.append(np.nan)
    Date.append(np.nan)
    comfort.append(np.nan)
    staff.append(np.nan)
    food.append(np.nan)
    entertainment.append(np.nan)
    wifi.append(np.nan)
    ground_service.append(np.nan)
    vfm.append(np.nan)
    recommend.append(np.nan)
    
    for row in soup.find_all('tr'):
        header=row.find('td',class_='review-rating-header').text
        value=row.find('td',class_='review-value')
        if header=='Aircraft':
            Aircraft[-1]=value.text
        elif header=='Type Of Traveller':
            Travel_type[-1]=value.text
        elif header=='Seat Type':
            Class[-1]=value.text
        elif header=='Route':
            Route[-1]=value.text
        elif header=='Date Flown':
            Date[-1]=value.text
        elif header=='Seat Comfort':
            comfort[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Cabin Staff Service':
            staff[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Food & Beverages':
            food[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Inflight Entertainment':
            entertainment[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Ground Service':
            ground_service[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Value For Money':
            vfm[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Recommended':
            recommend[-1]=value.text
        elif header=='Wifi & Connectivity':
            wifi[-1]=len(row.find_all('span',class_='star fill'))

In [4]:
def fetcher(soup,tag,key,value):
    if soup.find(tag,{key:value}):
        return soup.find(tag,{key:value}).text
    return np.nan

In [6]:
from webdriver_manager.chrome import ChromeDriverManager

def get_airline_names(url):
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get(url)
    time.sleep(3)
    
    ul_elements = driver.find_elements(By.XPATH,'/html/body/div[1]/div[1]/div/div/section[2]/div[1]/section/ul')
    list_ = []
    
    for ul in ul_elements:
        try:
            # Use find_elements to find all <div class="body"> elements within each ul
            body_divs = ul.find_elements(By.CLASS_NAME, 'body')
            
            # Iterate over the body_divs to find the <a> element and extract its text content
            for body_div in body_divs:
                airline_name = body_div.find_element(By.TAG_NAME, 'a').text
                
                # Append the data to the list
                list_.append(airline_name)
        except Exception as e:
            # Handle exceptions, print them or log them if needed
            print(f"Error: {e}")
            continue
            
    return list_
    


In [7]:
airline_list = get_airline_names('https://www.airlinequality.com/airline-reviews')
print(airline_list)

['Aeromexico', 'Air Canada', 'Air India', 'Allegiant Air', 'American Airlines', 'Avianca', 'Azerbaijan Airlines', 'Bluebird Airways', 'Breeze Airways', 'British Airways', 'Brussels Airlines', 'Cebu Pacific', 'China Airlines', 'Emirates', 'Flair Airlines', 'Frontier Airlines', 'IndiGo', 'ITA Airways', 'Japan Airlines', 'KLM Royal Dutch Airlines', 'LATAM Airlines', 'Lufthansa', 'Lynx Air', 'Pegasus Airlines', 'Philippine Airlines', 'PLAY', 'Porter Airlines', 'Qatar Airways', 'Royal Air Maroc', 'Ryanair', 'Scoot', 'Swiss Intl Air Lines', 'TAP Portugal', 'Thai Airways', 'Transavia', 'Turkish Airlines', 'United Airlines', 'Virgin Australia', 'Vistara', 'Vueling Airlines', 'WestJet Airlines']


In [8]:
for airline in airline_list:
    print(f"Scraping from: {airline}")
    for i in range(1, 10):
        print(f"Page {i}")
        url = f'https://www.airlinequality.com/airline-reviews/{airline}/page/{i}/'
        page = rq.get(url).text
        # Getting all the html content of the page
        soup = BeautifulSoup(page, 'lxml')
        articles = soup.find_all('article', {'itemprop': 'review'})
        # Iterating through each one of the article and extracting all the relevant infotmation like ratings
        for article in articles:
            airline_name.append(airline)
            rating.append(fetcher(article, 'span', 'itemprop', 'ratingValue'))
            div = article.find('div', class_='tc_mobile')
            content = div.find('div', class_='text_content').text.split('|')
            # handling the different formats, if the length of the contetn is 2, it contains the verification and review else we
            # set the verification to null as not all review has a verification status
            if len(content) == 2:
                verification, review = content
            else:
                verification = np.nan
                review = content[0]
            verified.append(verification)
            feedback.append(review)
            table = div.find('table', class_='review-ratings')
            value_fetcher(table)

Scraping from: Aeromexico
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Scraping from: Air Canada
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Scraping from: Air India
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Scraping from: Allegiant Air
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Scraping from: American Airlines
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Scraping from: Avianca
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Scraping from: Azerbaijan Airlines
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Scraping from: Bluebird Airways
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Scraping from: Breeze Airways
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Scraping from: British Airways
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Scraping from: Brussels Airlines
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 

In [9]:
data=pd.DataFrame({'status':verified,'aircraft':Aircraft,'travel_type':Travel_type,'travel_class':Class,'route':Route,'date':Date,'seating_comfort':comfort,'staff_service':staff,'food_quality':food,'entertainment':entertainment,'wifi':wifi,'ground_service':ground_service,'value_for_money':vfm,'recommended':recommend,'overall_rating':rating[::2],'review':feedback, 'airline_name': airline_name})

In [12]:
data.to_csv('raw_scraped_date_junming.csv',index=False)

In [13]:
data

,status,aircraft,travel_type,travel_class,route,date,seating_comfort,staff_service,food_quality,entertainment,wifi,ground_service,value_for_money,recommended,overall_rating,review,airline_name
0,Not Verified,NaN,Couple Leisure,Economy Class,Vancouver to Cancun,January 2024,2.0,3.0,2.0,3.0,1.0,2.0,2,no,1,Both my connecting flights from Mexico City t...,Aeromexico
1,✅ Trip Verified,Boeing 737 Max,Couple Leisure,Economy Class,Mexico City to Vancouver,January 2024,3.0,4.0,3.0,1.0,3.0,3.0,2,no,6,Uneventful flight on new years. Check in coul...,Aeromexico
2,✅ Trip Verified,Boeing 737-8 MAX,Couple Leisure,Economy Class,Mexico City to Toronto,December 2023,3.0,4.0,4.0,3.0,NaN,4.0,4,yes,8,Of all the North American carriers I have flo...,Aeromexico
3,✅ Trip Verified,Embraer 190,Couple Leisure,Economy Class,Queretaro to Mexico City,December 2023,3.0,4.0,NaN,NaN,NaN,4.0,3,yes,8,"What a pleasurable, although very short flig...",Aeromexico
4,✅ Trip Verified,Boeing 787,Solo Leisure,Economy Class,London to Mexico city,January 2024,3.0,1.0,1.0,4.0,1.0,4.0,2,no,5,Everything was just average. The ground staf...,Aeromexico
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,✅ Trip Verified,Boeing 737 max 8,Business,Economy Class,Calgary to Hamilton,January 2024,3.0,1.0,2.0,1.0,NaN,1.0,1,no,3,Absolute worst staff I have ever had to deal...,WestJet Airlines
1094,✅ Trip Verified,Boeing 737 max 8,Business,Economy Class,Calgary to Hamilton,January 2024,3.0,1.0,2.0,1.0,NaN,1.0,1,no,3,Absolute worst staff I have ever had to deal...,WestJet Airlines
1095,✅ Trip Verified,Boeing 737 max 8,Business,Economy Class,Calgary to Hamilton,January 2024,3.0,1.0,2.0,1.0,NaN,1.0,1,no,3,Absolute worst staff I have ever had to deal...,WestJet Airlines
1096,✅ Trip Verified,Boeing 737 max 8,Business,Economy Class,Calgary to Hamilton,January 2024,3.0,1.0,2.0,1.0,NaN,1.0,1,no,3,Absolute worst staff I have ever had to deal...,WestJet Airlines
